importing necessary libraries

In [68]:
from googleapiclient.discovery import build
import pandas as pd

Collecting the channel ids

In [69]:
api_key ='XXXX' #generate your own API key and add here
channel_ids = ['UCeVMnSShP_Iviwkknt83cww',# codewithharry
             'UCBwmMxybNva6P_5VmxjzwqA', # apna college
             'UCnz-ZXXER4jOvuED5trXfEA', # techTFQ
             'UCmDgY7fZKvgatlO0Tib2ggQ', # buzzfeed india
             'UCbuj4kbjP05NLiWUbpSuBPw', # monkey magic
               # more channel ids here
             ]


# Get credentials and create an API client
youtube=build('youtube','v3',developerKey=api_key)

## Function to get channel statistic


In [70]:
def get_channel_stats(youtube, channel_ids):
    all_data = []
    request = youtube.channels().list(
                part='snippet,contentDetails,statistics',
                id=','.join(channel_ids))
    response = request.execute() 
    
    for i in range(len(response['items'])):
        data = dict(Channel_name = response['items'][i]['snippet']['title'],
                    Subscribers = response['items'][i]['statistics']['subscriberCount'],
                    Views = response['items'][i]['statistics']['viewCount'],
                    Total_videos = response['items'][i]['statistics']['videoCount'],
                    playlist_id = response['items'][i]['contentDetails']['relatedPlaylists']['uploads'])
        all_data.append(data)
    
    return all_data
                                      

In [71]:
channel_statistics=get_channel_stats(youtube, channel_ids)

In [72]:
channel_data= pd.DataFrame(channel_statistics)

In [73]:
# changing the data type to numeric from object of certain columns
channel_data['Subscribers'] = pd.to_numeric(channel_data['Subscribers'])
channel_data['Views'] = pd.to_numeric(channel_data['Views'])
channel_data['Total_videos'] = pd.to_numeric(channel_data['Total_videos'])


In [74]:
# adding the data to a csv file
channel_data.to_csv('YT_Channel_details.csv')
#channel_data

In [75]:
#add the channel name for which you want to gather the data
playlist_id = channel_data.loc[channel_data['Channel_name']=='techTFQ', 'playlist_id'].iloc[0]

get video ids

In [76]:
def get_video_ids(youtube, playlist_id):
    
    request = youtube.playlistItems().list(
                part='contentDetails',
                playlistId = playlist_id,
                maxResults = 50)
    response = request.execute()
    
    video_ids = []
    
    for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])
        
    next_page_token = response.get('nextPageToken')
    more_pages = True
    
    while more_pages:
        if next_page_token is None:
            more_pages = False
        else:
            request = youtube.playlistItems().list(
                        part='contentDetails',
                        playlistId = playlist_id,
                        maxResults = 50,
                        pageToken = next_page_token)
            response = request.execute()
    
            for i in range(len(response['items'])):
                video_ids.append(response['items'][i]['contentDetails']['videoId'])
            
            next_page_token = response.get('nextPageToken')
        
    return video_ids

In [77]:
video_ids = get_video_ids(youtube, playlist_id)

In [78]:
#video_ids

get each video information

In [79]:
def get_video_details(youtube, video_ids):
    all_video_stats = []
    
    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
                    part='snippet,statistics',
                    id=','.join(video_ids[i:i+50]))
        response = request.execute()
         
        for video in response['items']:
            video_stats = dict(Video_id = video['id'],
                               Title = video['snippet']['title'],
                               Published_date = video['snippet']['publishedAt'],
                               Views = video['statistics']['viewCount'],
                               Likes = video['statistics']['likeCount'],
                               Comments = video['statistics']['commentCount'],
                               Channel_Title = video['snippet']['channelTitle']
                               )
            all_video_stats.append(video_stats)
        
    return all_video_stats
    #return response

In [80]:
video_details = get_video_details(youtube, video_ids)

In [81]:
video_data = pd.DataFrame(video_details)

In [82]:
video_data['Published_date'] = pd.to_datetime(video_data['Published_date']).dt.date
video_data['Views'] = pd.to_numeric(video_data['Views'])
video_data['Likes'] = pd.to_numeric(video_data['Likes'])
video_data['Comments'] = pd.to_numeric(video_data['Comments'])
video_data.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
video_data


,Video_id,Title,Published_date,Views,Likes,Comments,Channel_Title
0,7skZzocEU6c,Solving a Complex SQL Interview problem | Prac...,2023-03-21,19622,578,78,techTFQ
1,LZGaRcDxj8I,REAL SQL Interview Problem | Hierarchical data...,2023-03-14,10630,492,47,techTFQ
2,rBPQ5fg_kiY,Complete guide to Database Normalization in SQL,2023-03-07,21888,1177,177,techTFQ
3,hvwltYazuQo,SQL Live class on Lighthall | Learn SQL from B...,2023-01-26,10264,236,148,techTFQ
4,a-hFbr-4VQQ,How to learn SQL for free | Roadmap to learnin...,2022-12-28,225105,7727,315,techTFQ
...,...,...,...,...,...,...,...
82,1aybOgni7lI,How to install PostgreSQL on Mac OS | Install ...,2020-11-16,57087,580,92,techTFQ
83,j09EQ-xlh88,Learn What is Database | Types of Database | DBMS,2020-08-30,165789,3098,110,techTFQ
84,7nzTDrio7vY,Do you need a Smartwatch,2020-07-12,11213,182,46,techTFQ
85,J-uCLHTIWZ4,MacBook Pro 13 2020 One Week Later Review,2020-06-29,1338,70,20,techTFQ


creating csv file for the data collected of different channels

In [83]:
#(for each channel the upper function has to run seperately)

#video_data.to_csv('Apna_College_Video_details.csv')
#video_data.to_csv('Code_with_harry_Video_details.csv')
#video_data.to_csv('Buzzfeed_India_Video_details.csv')
#video_data.to_csv('Monkey_Magic_Video_details.csv')
#video_data.to_csv('techTFQ_Video_details.csv')

In [84]:
# merging csv files
df = pd.concat(map(pd.read_csv, ['Apna_College_Video_details.csv', 
                                 'Buzzfeed_India_Video_details.csv',
                                 'Code_with_harry_Video_details.csv',
                                 'Monkey_Magic_Video_details.csv',
                                 'techTFQ_Video_details.csv']),
               ignore_index=True)

df.to_csv('All_data.csv')

In [85]:
df['Year']=pd.DatetimeIndex(df['Published_date']).year
#df['Year']

In [86]:
#adding year column to MaindataFile.csv
csv_input = pd.read_csv('All_data.csv')
csv_input['Year'] =df['Year']
csv_input.head()
csv_input.to_csv("All_data.csv")

In [87]:
#removing unwanted columns

f=pd.read_csv("All_data.csv")
keep_col = ['Video_id','Title','Published_date','Views','Likes','Comments','Channel_Title','Year']
new_f = f[keep_col]
new_f.to_csv("MaindataFile.csv", index=False)